# Introduction
-----

Julia is a functional language.

All functions go right into the global namespace.

VS Code will use the local Project environment.
- Run `julia` in the project directory and type ] to enter package manger, then `activate .` to activate the env specified by the Project.toml in the repo. Generate a manifest by using `instantiate` (https://pkgdocs.julialang.org/v1/environments/#Using-someone-else's-project)\

Julia has extensive support for symbols, so it can be useful to change the default VS code fonts.

In [ ]:
# Exports QuantumCollocation, NamedTrajectories, and TrajectoryIndexingUtils
using Piccolo
using LinearAlgebra

# Plots
using CairoMakie

# Set up the Hamiltonian
-----

We will define some constants. Constants fix the type of variables, and let the compiler be more specific so code can run faster.

In [ ]:
const Units = 1e9
const MHz = 1e6 / Units
const GHz = 1e9 / Units
const ns = 1e-9 * Units
const μs = 1e-6 * Units

const n_qubits = 1
const n_levels = 2

t_f = 50 * ns
n_steps = 51
times = range(0, t_f, n_steps)  # Alternative: collect(0:Δt:t_f)
Δt = times[2] - times[1]

Julia defines column vectors by default. 
Compare columns:
- `x = [1, 2, 3]`
- `x = [1; 2; 3]`

Contrast with row:
- `x = [1 2 3]`

The imaginary is `im`

LinearAlgebra import an identity operator, `I`. This identity can rescale as necessary, but we can make it a concrete array type by constructing a matrix.

In [ ]:
# Operators
Paulis = Dict(
    "I" => Matrix{ComplexF64}(I, 2, 2),
    "X" => Matrix{ComplexF64}([0 1; 1 0]),
    "Y" => Matrix{ComplexF64}([0 -im; im 0]),
    "Z" => Matrix{ComplexF64}([1 0; 0 -1]),
)

A `QuantumSystem` contains the drift and control Hamiltonians. You can always check what methods exist for a function name by calling `methods`.

In [ ]:
methods(QuantumSystem)

It looks like we want `QuantumSystem(H_drift::Matrix{<:Number}, H_drives::Vector{<:Matrix{<:Number}}; ...)`

In [ ]:
# Add a tiny dephasing
Δω = .01 * GHz

H_drift = Δω * Paulis["Z"]
H_drives = [
    Paulis["X"],
    Paulis["Y"]
]

system = QuantumSystem(H_drift, H_drives)

You can explore the types that the functions return with a few useful tools.

In [ ]:
typeof(system)

In [ ]:
fieldnames(typeof(system))

And you can access the fields of the structs in the usual way

In [ ]:
system.H_drives_real

# Optimizaton problem
-----

Define a $\sqrt{X}$ gate. Notice that Julia knows you have a matrix, so it calls the correct sqrt function from linear algebra.


In [ ]:
# SX gate
target = sqrt([0 1; 1 0])

The adjoint can be computed with tick, and in that case multiplication can be suppressed. Let's check that our target is unitary.

In [ ]:
target'target 

In [ ]:
transpose(conj(target)) * target

The constructors are in `problem_templates.jl` from the package `QuantumCollocation.jl`. The problem templates make a problem that you can solve.

There are two main feature of the problem template: First, it constructs the dynamics constraint for you. Second, it constructs the objective function for you (fidelity, in our current case).

You always need to set some control bounds for the problem to be well-defined. 

We will also shape the cost with `Q` on the state norm and `R` on the control norm. You can be more precise with the `R` for the control positon, control velocity, and control acceleration.

The `hessian_approximation` is saying only use gradients; this is necessary for certain versions of the code for now, and doesn't matter too much if it's on.

`pade_order` tells how accurate you want the dynamics constraint to be. it goes from 4 to 20 in even numbers.


`free_time` and `timesteps_all_equal` can be set to do time optimization. I am fixing a bug for this currently, so you might not get the behavior you expect.

`subspace` isn't needed for this problem, but this will let you define qubit gates when using guard levels (extra transmon levels).

In [ ]:
PICO_max_iter = 100

# Shape the cost function with weights on states and controls
Q = 100.
R = .5

# Add control bounds
a_bound = 2 * π * 500 * MHz
dda_bound = .01

problem = UnitarySmoothPulseProblem(
    system,
    target,
    n_steps,
    Δt;
    a_bound=a_bound,
    dda_bound=dda_bound,
    Q=Q,
    R=R,
    verbose=true,
    hessian_approximation=false,
    pade_order=10,
    free_time=false,
    timesteps_all_equal=true,
    subspace=[1, 2],
    max_iter=PICO_max_iter,
)

Julia uses `!` to denote a function that changes the state of the arguments.

In [ ]:
solve!(problem)

If the optimization is slow, let me know! There may be some tricks we can play.

## Subspaces

`quantum_utils.jl` is where all the helpful tools live.

In [ ]:
# If you did need a subspace...
methods(subspace_indices)

In [ ]:
# Using transmons with 3 levels? Determine the indices you'd keep using...
# ...1 qubit
subspace_indices([3])

# ...2 qubits
subspace_indices([4, 4])

# Results
-----

The results of your problem are stored in a named trajectory.

In [ ]:
result = copy(problem.trajectory)

Let's use the pipe operator to check what the fieldnames are.

In [ ]:
result |> typeof |> fieldnames

Named trajectories are accessed using symbol names.

In [ ]:
typeof(:anything)

In [ ]:
result.names

The iso_vec_to_operator will take a unitary from a real and imaginary concatenation (iso) of the vectorized matrix (vec). This is how we solve the unitary dynamics. Under the hood, we map the unitary matrix to a vector.

The states are named using $\tilde{\vec{U}}$, which might not render well... See my comment at the top of the notebook about picking a VS code font like JuliaMono.

In [ ]:
iso_vec_to_operator(result[:Ũ⃗][:, 1])

We can map that function over the whole array.

In [ ]:
states = map(iso_vec_to_operator, eachslice(result[:Ũ⃗], dims=2))

Now we can rollout our controls using the dynamics model.

In [ ]:
methods(unitary_rollout)

In [ ]:
rollout_states = unitary_rollout(result, system; integrator=exp)

The rollouts don't need to match the states saved in the named trajectory.

The states in the named trajectory are the optimization variable, which only satisfy the dynamics up to the constraint.

You should always perform a rollout.

In [ ]:
# .- the dot applies the operation elementwise
ΔUs = map(norm, eachslice(rollout_states .- result[:Ũ⃗], dims=2))
maximum(ΔUs)

# Plot with Makie
-----

In [ ]:
ts = accumulate(+, timesteps(result)) .- timesteps(result)[1]
as = result[:a]

f = Figure()
ax = Axis(f[1, 1], xlabel="ns", ylabel="GHz")

lines!(ax, ts, as[1, :], label="X")
lines!(ax, ts, as[2, :], label="Y")

f